In [9]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana import system_program
from solana import transaction
from solana.system_program import SYS_PROGRAM_ID, CreateAccountParams, create_account
from solana.transaction import *
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import initialize_mint, InitializeMintParams, get_associated_token_address
from spl.token._layouts import MINT_LAYOUT
from spl.token.client import Token
from spl.token import instructions
import base64
import base58
from borsh_construct import *
from solana.rpc.commitment import Commitment
import time
import sys
sys.path.append('../../GlobalRepo/')
from global_repo import *


client = Client("https://api.devnet.solana.com")

client.is_connected()

True

In [10]:
program_id = GOVERNOR_PROGRAM_ID
# program_id_account_meta = AccountMeta(program_id, False, False)
sys_program_meta = AccountMeta(SYS_PROGRAM_ID, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)
# program_id_account_meta

In [11]:
PublicKey.find_program_address([b"Governor_Vault"], program_id)

(25N4Q6k5PPFmeqAco84y7UuM1w9bNmyJdssoJLJ9GqXv, 252)

In [12]:
def getAccountInfo(client: Client, account_pk: PublicKey):
    """Fetch account information from RPC, parse out the data and deserialize"""
    comm = Commitment("confirmed")
    res = client.get_account_info(account_pk, comm, encoding='base64')
    data = res['result']
    if isinstance(data, dict):
        return accountSchema.parse(base64.urlsafe_b64decode(data['value']['data'][0]))
    else:
        raise AttributeError(f'Unknown RPC result {data}')

In [13]:
seed = bytes([125,215,212,54,213,98,45,123]*4)
payer_keypair = payer_keypair = Keypair.from_secret_key(base58.b58decode("mH4PkhAttpFXSrntbpiJgC8J6wHYFDK5PFWcPN53hMJKziWQucux7MFiW7uXnqKsAY3XBnxUtyVKnomZnD9JxHW"))
payer_keypair.public_key

2ZHc9QxDDaJwqNEFzpAGUrvxCWZSNnXSffHxV9hG2axp

In [14]:
mint_account = Keypair.generate()

payer_account_meta = AccountMeta(payer_keypair.public_key, True, True)
vault_pda, _ = PublicKey.find_program_address([b"Governor_Vault"], program_id)

rent_account_meta = AccountMeta(solana.sysvar.SYSVAR_RENT_PUBKEY, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)

avatar_mint = PublicKey("8Z7vfkpxENuAV1joZZZDLmQmkUspgJuMhkLiUbq6jcjT")
avatar_program = AVATAR_PROGRAM_ID

In [15]:
mint_account_meta = AccountMeta(mint_account.public_key, True, True)
associated_token_address = instructions.get_associated_token_address(payer_keypair.public_key, mint_account.public_key)
associated_account_meta = AccountMeta(associated_token_address, False, True)

(mint_authority, nonce) = PublicKey.find_program_address([b'mint_authority'], program_id)
(avatar_data_pda, avatar_pda_nonce) = PublicKey.find_program_address([b'governor_data_pda', bytes(mint_account_meta.pubkey)], program_id)
(sales_pda, sales_nonce) = PublicKey.find_program_address([b'sales_pda', bytes(program_id)], program_id)
metadata_program_pubkey = METAPLEX_PROGRAM_ID
(metadata_pda, metadata_nonce) = PublicKey.find_program_address([b'metadata', bytes(metadata_program_pubkey), bytes(mint_account_meta.pubkey)], metadata_program_pubkey)
(avatar_metadata_pda, metadata_nonce) = PublicKey.find_program_address([b'metadata', bytes(metadata_program_pubkey), bytes(avatar_mint)], metadata_program_pubkey)
avatar_creator , _ = PublicKey.find_program_address([b"avatar_mint_authority_pda"], avatar_program)
avatar_associated_account = instructions.get_associated_token_address(payer_keypair.public_key, avatar_mint)
vault_account_meta = AccountMeta(vault_pda, False, True)


avatar_data_pda_meta = AccountMeta(avatar_data_pda, False, True)
sysvar_clock_meta = AccountMeta(solana.sysvar.SYSVAR_CLOCK_PUBKEY, False, False)
sales_pda_meta = AccountMeta(sales_pda, False, True)
mint_authority_meta = AccountMeta(mint_authority, False, False)

accounts = [
    payer_account_meta, 
    vault_account_meta, 
    mint_account_meta, 
    rent_account_meta, 
    associated_account_meta,
    spl_program_meta,
    AccountMeta(SYS_PROGRAM_ID, False, False),
    mint_authority_meta,
    avatar_data_pda_meta,
    sales_pda_meta,
    AccountMeta(metadata_pda, False, True),
    sysvar_clock_meta,
    AccountMeta(avatar_mint, False, False),
    AccountMeta(avatar_creator, False, False),
    AccountMeta(avatar_metadata_pda, False, False),
    AccountMeta(avatar_associated_account, False, False),

    spl_program_meta,
    AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
    spl_program_meta,
    spl_program_meta,
    AccountMeta(metadata_program_pubkey, False, False),
    ]
 
(vault_account_meta.pubkey, mint_authority, mint_account.public_key)

(25N4Q6k5PPFmeqAco84y7UuM1w9bNmyJdssoJLJ9GqXv,
 BxHRC3Qu6x4ieKr9ML4i3ZgBFC2riWELG35g63jJ2ed4,
 FGmgjsekE1TDYHAsCjv4HDrigWnpaWMdWogH9d1emdEh)

In [17]:
    
transaction = Transaction()
transaction.add(TransactionInstruction(
    accounts,
    program_id,
    bytes([0])
))
f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, mint_account)['result']}?cluster=devnet"

'https://explorer.solana.com/tx/4fN5BBVfSpuSE5f27d4Vgmn46G8JSQRRWokG7uVDjT1JPsvPzvmuyFmvmKErko4spiftcLP9naNToz3RfYrqASto?cluster=devnet'

In [18]:
time.sleep(2)
accountSchema = CStruct(
    "vault_total" / U64,
    "counter" / U32,
)

(sales_pda, sales_nonce) = PublicKey.find_program_address([b'sales_pda', bytes(program_id)], program_id)
print(f"PDA: {sales_pda, sales_nonce}")

getAccountInfo(client,sales_pda)

PDA: (ACfcMNwkWCZtLUTK8KQVj65a4dJQds1LyjX5as3JqvH7, 253)


Container(vault_total=12101955410, counter=8)

In [19]:
accountSchema = CStruct(
    "date_created" / U32,
    "unlockable_date" / U32,
    "numeration" /U32,
    "rarity" / U8,
    "level" / U8,
    "xp" / U32,
)


(locktime_pda, locktime_nonce) = PublicKey.find_program_address([b'locktime_pda', bytes(mint_account_meta.pubkey), bytes(associated_account_meta.pubkey)], program_id)
print(f"PDA: {avatar_data_pda, avatar_pda_nonce}")

getAccountInfo(client,avatar_data_pda)

PDA: (93eicXRdmec4DkPDJdJjhjBP2mBVHXKZJfc1f24ookzh, 251)


Container(date_created=1646588038, unlockable_date=1646588038, numeration=7, rarity=2, level=0, xp=0)

In [20]:
time.sleep(100)

new_accounts = [
    mint_account_meta,
    associated_account_meta,
    spl_program_meta,
    mint_authority_meta,
    avatar_data_pda_meta,
    sysvar_clock_meta,

    spl_program_meta
]

transaction = Transaction()
transaction.add(TransactionInstruction(
    new_accounts,
    program_id,
    bytes([1])
))
f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, mint_account)['result']}?cluster=devnet"

'https://explorer.solana.com/tx/4bZGCrUpbbRceBY4Gr4LW9nZTgs64TWKyDySbeB3tCsmvQN2YheXe1VKHin8d2mPy6js2zM36Bz5FpXvKupJcLVV?cluster=devnet'